## 1. Set parameters for queries
This step will open a dropdown so you can choose parameters for the following queries. 

In [25]:
import ipywidgets as widgets
from ipywidgets import HBox

# all available parameters
tasks = [('pour',"pour:Pouring"),
        ('drain', "pour:Draining"),
        ('sprinkle',"pour:Sprinkling")]

objects=[('batter', "obo:FOODON_03301304"),
        ('broth', "obo:FOODON_03420170"),
        ('chocolate', "obo:FOODON_03307240"),
        ('coffee', "obo:FOODON_03301036"),    
        ('cream', "pour:cream"),
        ('egg', "obo:FOODON_03316061"),
        ('flour', "obo:FOODON_03302339"),     
        ('juice', "obo:FOODON_03315552"), 
        ('liquids', "pour:Liquids"),
        ('milk', "obo:UBERON_0001913"),
        ('oil', "obo:FOODON_03310387"),
        ('sauce', "obo:FOODON_03311146"),
        ('sugar', "obo:FOODON_03301073"),
        ('syrup', "obo:FOODON_03303225"), 
        ('tea', "obo:FOODON_03315081"),
        ('vinegar', "obo:FOODON_03301705"),
        ('water', "obo:ENVO_00003064"),
        ('wine', "obo:FOODON_03301338")]

task=""
tobject=""

# Takes a Task of the tasks list. 
# Then makes the previously initialized task variable global and assigns value of the given Task to it 
def chooseTask(Task):
    global task
    task=Task
    
# Takes an Object of the object list.
# Then makes the previously initialized tobject variable global and assigns value of the given Object to it 
def chooseObject(Object):
    global tobject
    tobject=Object
    
# Create the dropdown widgets
task_widget = widgets.Dropdown(options=tasks, description='Task:')
object_widget = widgets.Dropdown(options=objects, description='Object:')

# Define the event handlers
def taskEvent(event):
    chooseTask(event.new)

def objectEvent(event):
    chooseObject(event.new)
    
# Attach the event handlers to the widgets
task_widget.observe(taskEvent, names='value')
object_widget.observe(objectEvent, names='value')

# Combine the widgets using HBox
widgets_display = HBox([task_widget, object_widget])

# Display widgets
widgets_display

# optional: set Parameters manually:
#task = "cut:Quartering"
#tobject = "obo:FOODON_03301710"

## 2. import rdflib for querying the knowledge graph for action parameters
This step is needed to load libraries for execution of the queries

In [27]:
!pip install rdflib
# import rdflib
from rdflib import Graph, Literal, URIRef, Namespace
# rdflib knows about quite a few popular namespaces, like W3C ontologies, schema.org etc
from rdflib.namespace import OWL, RDF, RDFS

--------------------------------------------------------------------------------------------------------------------------------
# Now let us start with querying the knowledge graph!
## Query 1: We can query for the <b>tool</b> to be used for the chosen task:
With this query, a robot can determine if a tool shall be used for the pouring task

In [32]:
import requests

url = "https://raw.githubusercontent.com/Food-Ninja/PouringFood/main/owl/PouringFood.owl"

response = requests.get(url)

g = Graph()
g.parse(data=response.content, format='xml')

# define prefixes to be used in the query 
SOMA = Namespace("http://www.ease-crc.org/ont/SOMA.owl#")
POUR = Namespace("http://www.ease-crc.org/ont/pouring_food#")
DUL = Namespace("http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#")
OBO = Namespace("http://purl.obolibrary.org/obo/")
QUDT = Namespace("http://qudt.org/schema/qudt#")
g.bind("owl", OWL)
g.bind("rdfs", RDFS)
g.bind("soma", SOMA)
g.bind("pour", POUR)
g.bind("dul", DUL)
g.bind("obo", OBO)
g.bind("qudt",QUDT)

# the actual query comes here
query = """SELECT ?whattool WHERE {
      %s rdfs:subClassOf ?n.
      ?n owl:onProperty dul:hasParticipant.
      ?n owl:someValuesFrom ?whattool.} """ % (task) # the input parameter for the query. For testing, you can remove % (tobject) here
#and replace %s in the first line of the query with a variable like ?food to get all food and objects

for row in g.query(query):
    print(row.whattool)

# Query 2: The robot needs to know the action parameters:
## First, let us query the needed <b>angle</b> for pouring

In [31]:
anglequery = """SELECT ?min ?max WHERE {  
  %s rdfs:subClassOf ?s.
  ?s owl:onProperty pour:hasConsistency.
  ?s owl:someValuesFrom ?consistency.
  ?union rdf:first*/rdf:rest* ?consistency.
  {
  ?rest rdf:first*/rdf:rest* ?union.
  ?start owl:unionOf ?rest.
  ?a owl:someValuesFrom ?start.
  }
  UNION
  {
    ?a owl:allValuesFrom ?union.
  }
  ?f rdf:first*/rdf:rest* ?a.
  ?r rdf:first*/rdf:rest* ?f.
  ?i owl:intersectionOf ?r.
  ?inter owl:someValuesFrom ?i.
  ?param rdfs:subClassOf ?inter.
  ?param rdfs:subClassOf pour:PouringAngle.
  ?param rdfs:subClassOf ?degree1.
  ?degree1 owl:onProperty qudt:valueQuantity.
  ?degree1 owl:minQualifiedCardinality ?min.
  ?param rdfs:subClassOf ?degree2.
  ?degree2 owl:onProperty qudt:valueQuantity.
  ?degree2 owl:maxQualifiedCardinality ?max. }""" % (tobject) # the input parameter for the query. For testing, you can remove % (tobject) here
#and replace %s in the first line of the query with a variable like ?food to get all food and objects

for row in g.query(anglequery):
    print("min angle:",row.min, "max angle:", row.max)

min angle: 1 max angle: 45


## Now let us examine how long the task should be performed

In [33]:
durationquery = """SELECT ?min ?max WHERE {  
  %s rdfs:subClassOf ?s.
  ?s owl:onProperty pour:hasConsistency.
  ?s owl:someValuesFrom ?consistency.
  ?union rdf:first*/rdf:rest* ?consistency.
  {
  ?rest rdf:first*/rdf:rest* ?union.
  ?start owl:unionOf ?rest.
  ?a owl:someValuesFrom ?start.
  }
  UNION
  {
    ?a owl:allValuesFrom ?union.
  }
  ?f rdf:first*/rdf:rest* ?a.
  ?r rdf:first*/rdf:rest* ?f.
  ?i owl:intersectionOf ?r.
  ?inter owl:someValuesFrom ?i.
  ?param rdfs:subClassOf ?inter.
  ?param rdfs:subClassOf pour:PouringDuration.
  ?param rdfs:subClassOf ?degree1.
  ?degree1 owl:onProperty qudt:valueQuantity.
  ?degree1 owl:minQualifiedCardinality ?min.
  ?param rdfs:subClassOf ?degree2.
  ?degree2 owl:onProperty qudt:valueQuantity.
  ?degree2 owl:maxQualifiedCardinality ?max. }""" % (tobject) # the input parameter for the query. For testing, you can remove % (tobject) here
#and replace %s in the first line of the query with a variable like ?food to get all food and objects

for row in g.query(durationquery):
    print("min duration:",row.min, "max duration:", row.max)

min duration: 1 max duration: 3
